# Import Basic Libraries


In [1]:
import numpy as np 
import pandas as pd 


# Import ML Focused Libraries


In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [38]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [9]:
def df_maker(data):
    df = pd.read_csv(data)
    return df 


In [7]:
%pwd


'c:\\Users\\pgsin\\Documents\\AI_ENGINEERING\\Striving\\M5_Deep_Learning\\LSTMS'

In [11]:
df = df_maker("TSLA.csv")

In [18]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-07-10,52.444000,52.599998,51.563999,51.830002,51.830002,13054500
1,2015-07-13,52.450001,52.509998,51.209999,52.431999,52.431999,14801500
2,2015-07-14,52.419998,53.198002,52.102001,53.130001,53.130001,9538000
3,2015-07-15,53.348000,53.498001,52.416000,52.627998,52.627998,10108000
4,2015-07-16,52.844002,53.439999,52.632000,53.335999,53.335999,8080000
...,...,...,...,...,...,...,...
1478,2021-05-24,581.599976,614.479980,573.650024,606.440002,606.440002,34558100
1479,2021-05-25,607.309998,613.989990,595.710022,604.690002,604.690002,28005900
1480,2021-05-26,607.559998,626.169983,601.500000,619.130005,619.130005,28639300
1481,2021-05-27,620.239990,631.130005,616.210022,630.849976,630.849976,26370600


In [29]:

msk = round(0.8 * len(df))
df_train = df.iloc[:msk,1:2].values
df_test = df.iloc[msk + 1 :  ,1:2 ].values


In [37]:
df_train.shape

(1186, 1)

In [30]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(df_train)

In [35]:
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [36]:
X_train.shape

(1126, 60, 1)

In [42]:
# n_steps is the size of the contiguous sequence that is taken from the TS
# it coincides with the number of time steps for the RNN
n_steps = 10

# An RNN can take as inputs multiple input sequences (e.g, a matrix of n_seq x n_steps)
n_inputs = 7

# There is one layer of X recurrent neurons together
n_neurons = 100

# The ouput will be a single sequence
n_outputs = 1

In [39]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [40]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [41]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 32)